## Assignment #2 - Carbon Stock Estimation

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from vol_eqs import VollibWrapper
from IPython.display import Markdown as md
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

# IMPORTANT: Update this path to your vollib DLL location
DLL_PATH = r"S:/projects/Fall2025/ForestAnalytics/ForestAnalytics/vol-lib-dll-20250701/VolLibDll20250701/vollib64/vollib.dll"

print("=== Forest Inventory Analysis Using VOLLIB ===")
print("Analysis of Lubrecht Experimental Forest Plot 9\n")

# Initialize the VOLLIB wrapper
try:
    vol_estimator = VollibWrapper(dll_path=DLL_PATH)
    print("✓ VOLLIB DLL loaded successfully")
except Exception as e:
    print(f"✗ Error loading VOLLIB DLL: {e}")
    print("Please update the DLL_PATH variable to match your system")
    raise

# Load the plot data from CSV file
try:
    plot_data = pd.read_csv('lef_plot_sample.csv', sep='\t')
    species_codes = pd.read_csv('fia_treenames.csv')
    plot_data = plot_data.merge(species_codes[['FIA Code', 'Common Name']], left_on='Species', right_on='FIA Code', how='left')
    plot_data.rename(columns={'Common Name': 'SpeciesName'}, inplace=True)
    print("Plot data loaded successfully")
    print(f"Dataset contains {len(plot_data)} trees")
except FileNotFoundError:
    print("CSV file 'lef_plot_sample.csv' not found")

=== Forest Inventory Analysis Using VOLLIB ===
Analysis of Lubrecht Experimental Forest Plot 9

✓ VOLLIB DLL loaded successfully
Plot data loaded successfully
Dataset contains 1203 trees


In [19]:
def calculate_tree_values(row):
    """Calculates values and BA for a single tree row."""
    try:
        params = {
            "region": 1, "forest": "16", "district": "01",
            "species_code": int(row['Species']),
            "dbh": row['DBH'], "height": row['TotalLen'],
        }
        params_temp = params.copy()
        # remove dbh and height for equation retrieval
        params_temp.pop('dbh', None)
        params_temp.pop('height', None)
        voleq = vol_estimator._get_volume_equation(**params_temp, eq_type='biomass')
        total_vol = vol_estimator.calc_total_cubic_ft(**params)
        basal_area = 0.005454154 * (row['DBH'] ** 2)
        carbon_content = vol_estimator.calc_carbon_content(**params)
        
        return pd.Series({
            'basalAreasqft': basal_area, 'totVol': total_vol,
            'carbon': carbon_content, 'EQnum': voleq
        })
    except Exception:
        basal_area = 0.005454154 * (row['DBH'] ** 2)
        return pd.Series({
            'basalAreasqft': basal_area, 'totVol': 0,
            'carbon': 0, 'volEQnum': 'Error'
        })

volume_results = plot_data.apply(calculate_tree_values, axis=1)
plot_data = pd.concat([plot_data, volume_results], axis=1)

In [20]:
plot_data

,Year,PlotID,TreeID,unique_tree,group,Species,Status,StatusNote,DecayClass,DBH,...,DOB_break,Defect,Comments,FIA Code,SpeciesName,EQnum,basalAreasqft,carbon,totVol,volEQnum
0,2018,12,1,153,Survivor,202,1,NaN,NaN,24.8,...,NaN,NaN,NaN,202.0,Douglas-fir,NVBM330202,3.354523,2148.897217,94.700005,NaN
1,2018,12,2,154,Survivor,202,1,NaN,NaN,5.4,...,NaN,NaN,NaN,202.0,Douglas-fir,NVBM330202,0.159043,42.970219,2.100000,NaN
2,2018,12,3,155,Survivor,202,1,NaN,NaN,7.1,...,NaN,NaN,NaN,202.0,Douglas-fir,NVBM330202,0.274944,105.223106,5.300000,NaN
3,2018,12,4,156,Survivor,202,1,NaN,NaN,6.2,...,NaN,NaN,NaN,202.0,Douglas-fir,NVBM330202,0.209658,71.641937,3.600000,NaN
4,2018,12,5,157,Survivor,202,1,NaN,NaN,14.0,...,NaN,NaN,NaN,202.0,Douglas-fir,NVBM330202,1.069014,516.241699,23.800001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,2023,283,13,4182,Survivor,108,1,NaN,NaN,6.5,...,NaN,NaN,NaN,108.0,lodgepole pine,NVBM330108,0.230438,97.994141,5.900000,NaN
1199,2023,283,14,4183,Survivor,108,1,NaN,NaN,8.2,...,NaN,NaN,NaN,108.0,lodgepole pine,NVBM330108,0.366737,155.969299,8.800000,NaN
1200,2023,283,15,4184,Survivor,108,1,NaN,NaN,8.0,...,NaN,NaN,NaN,108.0,lodgepole pine,NVBM330108,0.349066,145.978516,8.200000,NaN
1201,2023,283,16,4654,Ingrowth_Live,108,1,A,NaN,5.1,...,NaN,NaN,NaN,108.0,lodgepole pine,NVBM330108,0.141863,51.328793,3.000000,NaN
